In [1]:
from ase.io import read
from ase.cluster import Octahedron
from ase.optimize import LBFGS
from ase.visualize import view
from mace.calculators import MACECalculator
from acat.ga.adsorbate_operators import (AddAdsorbate, RemoveAdsorbate,
                                        MoveAdsorbate, ReplaceAdsorbate,
                                        SimpleCutSpliceCrossoverWithAdsorbates)
from acat.ga.particle_mutations import (RandomPermutation, COM2surfPermutation,
                                        Rich2poorPermutation, Poor2richPermutation,
                                        RandomMutation)
from ase.ga.offspring_creator import OperationSelector
from acat.adsorption_sites import ClusterAdsorptionSites
from acat.adsorbate_coverage import ClusterAdsorbateCoverage
import numpy as np
from npl.monte_carlo.ensembles.canonical_ensemble import CanonicalEnsemble

/home/riccardo/anaconda3/envs/np-lib/lib/python3.9/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.lo

In [2]:
atoms = Octahedron('Ag', 6,1)
atoms.set_cell([25,25,25])
atoms.center()

sas = ClusterAdsorptionSites(atoms, ignore_sites=['ontop', 'bridge', 'longbridge', 'shortbridge'])
species = ['O']

add = AddAdsorbate(species, adsorption_sites=sas, num_muts=18, min_adsorbate_distance=2.7)
atoms.info['data'] = {'tag': None}
atoms.info['confid'] = 1
atoms, _ = add.get_new_individual([atoms])

view(atoms, viewer='x3d')

In [3]:
sas = ClusterAdsorptionSites(atoms, ignore_sites=['ontop', 'bridge', 'longbridge', 'shortbridge'])
species = ['O']
calculator = MACECalculator(
    model_paths='/home/riccardo/Downloads/mace_agnesi_medium.model',
    device = 'cpu'
    )

INFO - Using CPU


No dtype selected, switching to float64 to match model dtype.


In [ ]:
soclist = ([1, 100],[
    MoveAdsorbate(['O'], adsorption_sites=sas, num_muts=1),
    RandomMutation(0.02, num_muts=np.random.choice(80))
    ])
op_list = OperationSelector(*soclist)

outfile = f'{atoms.get_chemical_formula()}.out'
outfile_write_interval = 10
random_seed = 4234

montecarlo = CanonicalEnsemble(atoms=atoms,
                calculator=calculator,
                random_seed=random_seed,
                optimizer=LBFGS, 
                fmax=1000,
                temperature=600,
                p=1,
                traj_file='run1.traj', 
                op_list=op_list, 
                outfile=outfile,
                outfile_write_interval = outfile_write_interval)

montecarlo.run(steps=2000)

INFO - +---------------------------------+
INFO - | Canonical Ensemble Monte Carlo  |
INFO - +---------------------------------+
INFO - Starting simulation...
INFO - Temperature: 600
INFO - Number of steps: 2000
INFO - Step: 10
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 4
INFO - Step: 20
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 11
INFO - Step: 30
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 17
INFO - Step: 40
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 23
INFO - Step: 50
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 30
INFO - Step: 60
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 34
INFO - Step: 70
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 42
INFO - Step: 80
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 49
INFO - Step: 90
INFO - Lowest energy: -434.32593241698004
INFO - Accepted trials: 55
INFO - Step: 100
INFO - 